In [1]:
from array import array
import subprocess
import math
import ROOT
import os
import numpy as np

Welcome to JupyROOT 6.29/01


In [2]:
!pwd

/home/mdessole/Projects/ROOT/genvectorxfork/plots


In [3]:
buildcuda = '/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/'

In [4]:
buildoneapi = '/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/'

In [6]:
buildacpp = '/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/'

In [ ]:
# must be set before starting the kernel
os.environ['ACPP_VISIBILITY_MASK']='cuda'

In [11]:
!ls /home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [9]:
!ls /home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [12]:
!echo $LD_LIBRARY_PATH

/home/mdessole/Projects/AdaptiveCpp/AdaptiveCpp_install/lib:/usr/local/cuda-11.8/lib64:/home/mdessole/Projects/ROOT/root/lib:/opt/rocm-5.7.0/lib:/opt/rocm-5.7.0/opencl/lib:/home/mdessole/Projects/OneAPI/build/lib


In [13]:
sizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [14]:
Ssizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000,200000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [15]:
def result2list(exe, N, nruns):
    print(exe, str(int(N)), str(int(nruns)))
    result = subprocess.run([exe, str(int(N)), str(int(nruns)) ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    if len(result)<=0:
        return []
    result = result.replace("\r","").replace("\n","")
    time = np.zeros(nruns)
    for r in range(nruns):
        i = result.find("time")
        j = result.find("(s)")
        if (i<0) or (j<0):
            break
        time[r] = float(result[i+5:j])
        result = result[j+3:]
    #endif
    return time

In [16]:
def run_benchmark(builddir, sizes, nruns = 1, testname = "InvariantMasses", btype = 'cpu'):
    timeopt = array( 'd' )
    timestd = array( 'd' )
    
    if btype == 'cpu':
        exe = os.path.join(builddir,'testx',testname)
    elif btype == 'cuda':
        exe = os.path.join(builddir,'testx',testname+'CUDA')
    elif btype == 'sycl':
        exe = os.path.join(builddir,'testx',testname+'SYCL')
    #endif
    
    if (not os.path.exists(exe)):
        print("Executable", exe, "not found!")
        return timeopt, timestd
    
    for i, N in enumerate(sizes):
        time = result2list(exe,N,nruns)
        if (len(time)<=0):
            return timeopt, timestd
        print(time, '\n')
        timeopt.append(np.amin(time))
        timestd.append(np.std(time)) 
    #endfor
    return timeopt, timestd

In [17]:
timecpu, stdcpu = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000 3
RVec cpu time 6e-05 (s)
RVec cpu time 6e-05 (s)
RVec cpu time 6e-05 (s)

[6.e-05 6.e-05 6.e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 10000 3
RVec cpu time 0.000605 (s)
RVec cpu time 0.000606 (s)
RVec cpu time 0.000608 (s)

[0.000605 0.000606 0.000608] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 100000 3
RVec cpu time 0.006007 (s)
RVec cpu time 0.006108 (s)
RVec cpu time 0.006031 (s)

[0.006007 0.006108 0.006031] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000000 3
RVec cpu time 0.064251 (s)
RVec cpu time 0.061208 (s)
RVec cpu time 0.06122 (s)

[0.064251 0.061208 0.06122 ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 5000000 3
RVec cpu time 0.320346 (s)
RVec cpu time 0.322238 (s)
RVec cpu time 0.319997 (s)

[0.320346 0.322238 0.319997] 

/home/mdesso

In [18]:
timecpu

array('d', [6e-05, 0.000605, 0.006007, 0.061208, 0.319997, 0.641279, 3.21728, 6.47998])

In [19]:
Stimecpu, Sstdcpu = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000 3
SINGLE_PRECISION defined 
RVec cpu time 4.5e-05 (s)
RVec cpu time 4.5e-05 (s)
RVec cpu time 4.5e-05 (s)

[4.5e-05 4.5e-05 4.5e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 10000 3
SINGLE_PRECISION defined 
RVec cpu time 0.000445 (s)
RVec cpu time 0.00066 (s)
RVec cpu time 0.000624 (s)

[0.000445 0.00066  0.000624] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 100000 3
SINGLE_PRECISION defined 
RVec cpu time 0.004433 (s)
RVec cpu time 0.004435 (s)
RVec cpu time 0.004408 (s)

[0.004433 0.004435 0.004408] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000000 3
SINGLE_PRECISION defined 
RVec cpu time 0.046174 (s)
RVec cpu time 0.044463 (s)
RVec cpu time 0.044576 (s)

[0.046174 0.044463 0.044576] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 5000000 3
SINGLE_

In [20]:
Stimecpu

array('d', [4.5e-05, 0.000445, 0.004408, 0.044463, 0.233446, 0.462638, 2.34286, 4.69184, 9.28853])

In [21]:
timeoneapi, stdoneapi = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001165 (s)
sycl time 5.6e-05 (s)
sycl time 0.000145 (s)

[1.165e-03 5.600e-05 1.450e-04] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000536 (s)
sycl time 0.000108 (s)
sycl time 0.000205 (s)

[0.000536 0.000108 0.000205] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001549 (s)
sycl time 0.001223 (s)
sycl time 0.00105 (s)

[0.001549 0.001223 0.00105 ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000000

In [22]:
timeoneapi

array('d', [5.6e-05, 0.000108, 0.00105, 0.009915, 0.049908, 0.096058, 0.473085, 0.940488])

In [23]:
Stimeoneapi, Sstdoneapi = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00048 (s)
sycl time 4.4e-05 (s)
sycl time 0.000129 (s)

[4.80e-04 4.40e-05 1.29e-04] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000466 (s)
sycl time 0.000104 (s)
sycl time 0.000171 (s)

[0.000466 0.000104 0.000171] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00095 (s)
sycl time 0.000492 (s)
sycl time 0.000522 (s)

[0.00095  0.000492 0.000522] 

/home/mdessol

In [24]:
timeacpp, stdacpp = run_benchmark(buildacpp, sizes,  nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000773 (s)
sycl time 8.1e-05 (s)
sycl time 7.5e-05 (s)

[7.73e-04 8.10e-05 7.50e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000579 (s)
sycl time 0.000156 (s)
sycl time 0.000163 (s)

[0.000579 0.000156 0.000163] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001813 (s)
sycl time 0.001248 (s)
sycl time 0.001211 (s)

[0.001813 0.001248 0.001211] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/InvariantM

In [14]:
timeacpp

array('d', [0.000109, 0.00018, 0.001121, 0.011112, 0.050796, 0.095799, 0.46526, 0.928883])

In [25]:
Stimeacpp, Sstdacpp = run_benchmark(buildacpp, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000878 (s)
sycl time 9.5e-05 (s)
sycl time 9.4e-05 (s)

[8.78e-04 9.50e-05 9.40e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000495 (s)
sycl time 0.000115 (s)
sycl time 0.000116 (s)

[0.000495 0.000115 0.000116] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001002 (s)
sycl time 0.000518 (s)
sycl time 0.000511 (s)

[0.001002 0.000518 0.000511]

In [26]:
timecuda, stdcuda = run_benchmark(buildcuda, sizes, nruns = 3, btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000437 (s)
cuda time 4.8e-05 (s)
cuda time 5.1e-05 (s)

[4.37e-04 4.80e-05 5.10e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000458 (s)
cuda time 0.000185 (s)
cuda time 0.000169 (s)

[0.000458 0.000185 0.000169] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.00168 (s)
cuda time 0.001432 (s)
cuda time 0.001336 (s)

[0.00168  0.001432 0.001336] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.0125 (s)
cuda time 0.01194

In [27]:
timecuda

array('d', [4.8e-05, 0.000169, 0.001336, 0.011947, 0.060033, 0.114169, 0.559116, 1.10614])

In [28]:
Stimecuda, Sstdcuda = run_benchmark(buildcuda, Ssizes, nruns = 3, testname = "SInvariantMasses", btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.00039 (s)
cuda time 3e-05 (s)
cuda time 2.8e-05 (s)

[3.9e-04 3.0e-05 2.8e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000291 (s)
cuda time 9.4e-05 (s)
cuda time 8.9e-05 (s)

[2.91e-04 9.40e-05 8.90e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000987 (s)
cuda time 0.000698 (s)
cuda time 0.000702 (s)

[0.000987 0.000698 0.000702] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUD

In [29]:
n = len(sizes) 
speedupacpp = array('d', [timecpu[i]/timeacpp[i] for i in range(n)])
speeduponeapi   = array('d', [timecpu[i]/timeoneapi[i] for i in range(n)])
speedupcuda     = array('d', [timecpu[i]/timecuda[i] for i in range(n)])

In [30]:
n = len(Ssizes)
Sspeedupacpp = array('d', [Stimecpu[i]/Stimeacpp[i] for i in range(n)])
Sspeeduponeapi   = array('d', [Stimecpu[i]/Stimeoneapi[i] for i in range(n)])
Sspeedupcuda     = array('d', [Stimecpu[i]/Stimecuda[i] for i in range(n)])

In [31]:
print(Sspeedupacpp)
print(Sspeeduponeapi)
print(Sspeedupcuda)

array('d', [0.47872340425531923, 3.869565217391304, 8.626223091976517, 9.476342710997443, 10.373533594027728, 10.145125213806413, 10.467654667387487, 10.713333211552161, 10.828821581794442])
array('d', [1.022727272727273, 4.278846153846154, 8.959349593495933, 9.541416309012876, 10.221375716975349, 10.104355043026253, 10.767364159033773, 10.627068237980154, 10.623075049206689])
array('d', [1.6071428571428572, 5.0, 6.315186246418337, 8.919358074222668, 9.730565628777459, 9.963989575929874, 10.490809358558154, 10.367676142704042, 10.473206785547168])


In [32]:
val = list(sizes)
sizes_gb = [v*8*4/1e9 for v in val] #(nb of doubles)*(bytes for doubles)*(4=dim of LVector)/(bytes in GB)
sizes_gb = array('d', sizes_gb)
sizes_gb

array('d', [3.2e-05, 0.00032, 0.0032, 0.032, 0.16, 0.32, 1.6, 3.2])

In [33]:
val = list(Ssizes)
Ssizes_gb = [v*4*4/1e9 for v in val] #(nb of floats)*(bytes for floats)*(4=dim of LVector)/(bytes in GB)
Ssizes_gb = array('d', Ssizes_gb)
Ssizes_gb

array('d', [1.6e-05, 0.00016, 0.0016, 0.016, 0.08, 0.16, 0.8, 1.6, 3.2])

In [34]:
p = ROOT.TGraph(len(sizes), sizes, speedupacpp)
g = ROOT.TGraph(len(sizes), sizes, speeduponeapi)
q = ROOT.TGraph(len(sizes), sizes, speedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec")
q.GetXaxis().SetTitle( 'Input size (# doubles)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("speedup.png")

Info in <TCanvas::Print>: png file speedup.png has been created


In [53]:
len(Ssizes), Ssizes, len(Sspeedupacpp)

(9,
 array('d', [1000.0, 10000.0, 100000.0, 1000000.0, 5000000.0, 10000000.0, 50000000.0, 100000000.0, 200000000.0]),
 8)

In [35]:
p = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupacpp)
g = ROOT.TGraph(len(Ssizes), Ssizes, Sspeeduponeapi)
q = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec - Single Precision")
q.GetXaxis().SetTitle( 'Input size (# floats)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sspeedup.png")

Info in <TCanvas::Print>: png file Sspeedup.png has been created


In [37]:
p = ROOT.TGraphErrors(len(sizes), sizes, timeacpp, stdacpp)
g = ROOT.TGraphErrors(len(sizes), sizes, timeoneapi, stdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, timecuda, stdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, timecpu, stdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times")
r.GetXaxis().SetTitle( 'Input size (# doubles)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("exectimes.png")

Info in <TCanvas::Print>: png file exectimes.png has been created


In [38]:
p = ROOT.TGraphErrors(len(sizes), sizes, Stimeacpp, Sstdacpp)
g = ROOT.TGraphErrors(len(sizes), sizes, Stimeoneapi, Sstdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, Stimecuda, Sstdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, Stimecpu, Sstdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times - Single Precision")
r.GetXaxis().SetTitle( 'Input size (# floats)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sexectimes.png")

Info in <TCanvas::Print>: png file Sexectimes.png has been created
